In [1]:
import pandas as pd
import numpy as np
from math import sin,isnan
from pylab import *

In [2]:
df = pd.read_csv("D:/mins15_final_data-11839.csv")

In [15]:
df.drop(['Unnamed: 0'], axis = 1, inplace=True)

In [4]:
df2 = pd.read_csv("Cluster_wise_meters.csv")

In [8]:
c1 = df2[df2['clus_num'] == 1]
c2 = df2[df2['clus_num'] == 2]
c3 = df2[df2['clus_num'] == 3]
c4 = df2[df2['clus_num'] == 4]
c5 = df2[df2['clus_num'] == 5]
c6 = df2[df2['clus_num'] == 6]
c7 = df2[df2['clus_num'] == 7]
c8 = df2[df2['clus_num'] == 8]
c9 = df2[df2['clus_num'] == 9]
c10 = df2[df2['clus_num'] == 10]
c11 = df2[df2['clus_num'] == 11]
c12 = df2[df2['clus_num'] == 12]

In [27]:
#read all csv meters files
for index, row in c12.iterrows():
    no = row['idmeter']#.astype(str)
    df3 = df[df['idmeter'].str.contains(no+'$', regex=True)]
    df3.to_csv("ClusterCSV_data/Cluster 12/Cluster12_"+no+"_data.csv", index=False)

In [88]:
df6 = pd.DataFrame(columns=['idmeter', 'Average_load(KWh)', 'Morn_Peak_avarge_load(KWh)', 'Eve_Peak_avarge_load(KWh)','Peak_mor_7_10','Peak_eve_17_23','Total_peaks', 'Solar_size',
                           'Demand7to18day', 'Totaldemandday', 'Eligible_DSM','Eligible_NM','Eligible_DSM_NM'])

In [89]:
peak1 = 0
peak2 = 0
eligible_dsm = 0
eligible_nm = 0
eligible_dsm_nm = 0

In [90]:
for index, row in c12.iterrows():
    no = row['idmeter']
    df5 = pd.read_csv("ClusterCSV_data/Cluster 12/Cluster12_"+no+"_data.csv")
    df5.set_index('idmeter')
    df5['Average'] = df5.mean(axis=1)
    Average_load = (df5['Average'].mean(axis=0)) * 4
    Average_load_peaks = Average_load * 1.7
    df5['Eve_Peak Average'] = df5.iloc[:, [70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93]].mean(axis=1)
    Eve_Peak_average_load = (df5['Eve_Peak Average'].mean(axis=0)) * 4
    df5['Morn_Peak Average'] = df5.loc[:, '07:00':'09:45'].mean(axis=1)
    Morn_Peak_average_load = (df5['Morn_Peak Average'].mean(axis=0)) * 4
    #df5['Total_usage'] = df5.iloc[:, [30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74]].sum(axis=1)
    #Total_usage = (df5['Total_usage'].mean(axis=0))
    #To find the number of peaks - starts
    df5['peak_7_8'] = df5.loc[:, '07:00':'07:45'].sum(axis=1)
    peak_7_8 = (df5['peak_7_8'].mean(axis=0))
    df5['peak_8_9'] = df5.loc[:, '08:00':'08:45'].sum(axis=1)
    peak_8_9 = (df5['peak_8_9'].mean(axis=0))
    df5['peak_9_10'] = df5.loc[:, '09:00':'09:45'].sum(axis=1)
    peak_9_10 = (df5['peak_9_10'].mean(axis=0))
    df5['peak_17_18'] = df5.loc[:, '17:00':'17:45'].sum(axis=1)
    peak_17_18 = (df5['peak_17_18'].mean(axis=0))
    df5['peak_18_19'] = df5.loc[:, '18:00':'18:45'].sum(axis=1)
    peak_18_19 = (df5['peak_18_19'].mean(axis=0))
    df5['peak_19_20'] = df5.loc[:, '19:00':'19:45'].sum(axis=1)
    peak_19_20 = (df5['peak_19_20'].mean(axis=0))
    df5['peak_20_21'] = df5.loc[:, '20:00':'20:45'].sum(axis=1)
    peak_20_21 = (df5['peak_20_21'].mean(axis=0))
    df5['peak_21_22'] = df5.loc[:, '21:00':'21:45'].sum(axis=1)
    peak_21_22 = (df5['peak_21_22'].mean(axis=0))
    df5['peak_22_23'] = df5.loc[:, '22:00':'22:45'].sum(axis=1)
    peak_22_23 = (df5['peak_22_23'].mean(axis=0))
    if peak_7_8 > Average_load_peaks or peak_8_9 > Average_load_peaks or peak_9_10 > Average_load_peaks:
        peak1 = peak1 + 1
    if peak_17_18 > Average_load_peaks or peak_18_19 > Average_load_peaks or peak_19_20 > Average_load_peaks or peak_20_21 > Average_load_peaks or peak_21_22 > Average_load_peaks or peak_22_23 > Average_load_peaks:
        peak2 = peak2 + 1
    total_peaks = peak1+peak2
    #To find the number of peaks- ends
    #Find solar panel size - starts 
    Size_solar = ((df5.loc[:, '00:00':'23:45'].sum().sum())*0.9)/(0.1820 * 2952)
    #Find solar panel size - ends
    #Find the proportion of 7 to 6 (inclusive) demand in a day - starts
    totaldemandday = df5.loc[:,'00:00':'23:45'].mean().sum()
    #totaldemandday = df5['7to6demand'].sum()
    demand7to18day = df5.loc[:, '07:00':'18:00'].mean().sum()
    consump_7_18 = (demand7to18day/totaldemandday) * 100
    #Find the proportion of 7 to 6 (inclusive) demand in a day - ends
    if total_peaks >= 1:
        eligible_dsm = eligible_dsm + 1
    if consump_7_18 > 50 and Size_solar >= 1:
        eligible_nm = eligible_nm + 1
    if total_peaks >= 1 and consump_7_18 > 50 and Size_solar >= 1:
        eligible_dsm_nm = eligible_dsm_nm + 1
    df6 = df6.append({'idmeter': no, 'Average_load(KWh)': Average_load, 'Morn_Peak_avarge_load(KWh)': Morn_Peak_average_load, 'Eve_Peak_avarge_load(KWh)': Eve_Peak_average_load, 'Peak_mor_7_10': peak1, 
                      'Peak_eve_17_23': peak2,'Total_peaks': total_peaks, 'Solar_size': Size_solar, 'Demand7to18day': demand7to18day, 'Totaldemandday': totaldemandday,
                      'Eligible_DSM': eligible_dsm,'Eligible_NM': eligible_nm,'Eligible_DSM_NM': eligible_dsm_nm}, ignore_index=True)
    peak1 = 0
    peak2 = 0
    eligible_dsm = 0
    eligible_nm = 0
    eligible_dsm_nm = 0

In [91]:
df6.insert(loc=3, column='Morn_difference', value=df6['Morn_Peak_avarge_load(KWh)'] - df6['Average_load(KWh)'])
df6.insert(loc=5, column='Eve_difference', value=df6['Eve_Peak_avarge_load(KWh)'] - df6['Average_load(KWh)'])
df6['ConsumptionPerc7to18'] = (df6['Demand7to18day']/df6['Totaldemandday'])*100

In [92]:
df6.to_csv("Cluster_results/Cluster12_results.csv", index=False)

In [159]:
df_1 = pd.DataFrame(columns=['Cluster No', 'Morn_Perc_peak>avg', 'Eve_Perc_peak>avg', 'Peak_morn_7_10','Peak_eve_17_23', 'Total_peaks_0', 
                             'Total_peaks_1', 'Total_peaks_2', 'Solar size>=1', 'Proportion7to18Consu>60','EligibleDSM','EligibleNM','EligibleDSM_NM'])

In [94]:
for i in range(1,13):
    df_2 = pd.read_csv("Cluster_results/Cluster"+str(i)+"_statistics.csv")
    #df_2 = pd.read_csv("ClusterCSV_data/Cluster_statistics/Cluster1_statistics.csv")
    df_1 = df_1.append({'Cluster No': i, 'Morn_Perc_peak>avg': len(df_2.loc[df_2.Morn_difference > 0])*100 / df_2['idmeter'].nunique(),
                    'Eve_Perc_peak>avg': len(df_2.loc[df_2.Eve_difference > 0])*100 / df_2['idmeter'].nunique(),
                    'Peak_morn_7_10': len(df_2.loc[df_2.Peak_mor_7_10 == 1])*100 / df_2['idmeter'].nunique(),
                    'Peak_eve_17_23': len(df_2.loc[df_2.Peak_eve_17_23 == 1])*100 / df_2['idmeter'].nunique(),
                    'Total_peaks_0': len(df_2.loc[df_2.Total_peaks == 0])*100 / df_2['idmeter'].nunique(),
                    'Total_peaks_1': len(df_2.loc[df_2.Total_peaks == 1])*100 / df_2['idmeter'].nunique(),
                    'Total_peaks_2': len(df_2.loc[df_2.Total_peaks == 2])*100 / df_2['idmeter'].nunique(),
                    'Solar size>=1': (len(df_2.loc[df_2['Solar_size'] >= 1]))*100 / df_2['idmeter'].nunique(),
                    'Proportion7to18Consu>60':(len(df_2.loc[df_2['ConsumptionPerc7to18'] > 50]))*100 / df_2['idmeter'].nunique(),
                    'EligibleDSM': len(df_2.loc[df_2.Eligible_DSM == 1])*100 / df_2['idmeter'].nunique(),
                    'EligibleNM': len(df_2.loc[df_2.Eligible_NM == 1])*100 / df_2['idmeter'].nunique(),
                    'EligibleDSM_NM': len(df_2.loc[df_2.Eligible_DSM_NM == 1])*100 / df_2['idmeter'].nunique(),}, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Cluster_statistics/Cluster1_statistics.csv'

In [158]:
df_1.to_csv("Cluster_results/All_clusters_results.csv", index=False)